In [ ]:
### 只做单模型，其他的内容，等等再说，现在把单模型的表现提升上去再说吧

In [1]:
import sys
sys.path.append("../")
import pandas as pd
import numpy as np
from dateutil.parser import parse
from sklearn import metrics
import datetime

import model_ml as mm
import feat_engineering as fe
import feat_selection as fs
from param_config import config

In [2]:
dfTrain = pd.read_csv(config.original_train_data_path)
dfPred = pd.read_csv(config.original_pred_data_path)
predictors = dfPred.columns.tolist()[4:]
check_missing = ['PartI_1','PartII_1','PartIII_1','PartIV_1','PartV_1','PartVI_1']


###清理异常Y值
dfTrain = dfTrain.loc[dfTrain['Y']<dfTrain['Y'].max()]
dfTrain = dfTrain.reset_index(drop=True)

In [3]:
def FeatAll(train,pred):
    dfAll = pd.concat([train,pred])
    dfAll = dfAll.reset_index(drop=True)
    dfAll['date'] = (pd.to_datetime(dfAll['date']) - parse('2017-10-09')).dt.days

    ###这个部分将每一列分箱，并且计算基于该列分类的其他列的百分比顺序，目前看来没啥用
    var_dict = {}
    for colBase in predictors+['sex','age']:
        if dfAll[colBase].nunique()<=5:
            var_dict[colBase] = [x for x in predictors if x!=colBase]
        else:
            dfAll[colBase+'_based'] = pd.cut(dfAll[colBase],bins=5,labels=[colBase+'_bin_%d'%i for i in range(5)])
            dfAll[colBase+'_based'] = dfAll[colBase+'_based'].astype(str)
            dfAll[colBase+'_based'].fillna(colBase+'_bin_NA',inplace = True)
            var_dict[colBase+'_based'] = [x for x in predictors if x!=colBase]
    #dfAllPcentByOther = fe.pcent_by_other_col(dfAll,var_dict,['ID']) 
    #dfAllPcentByOther = pd.read_csv('../../Cache/pcent_by_other.csv')
    
    for var in dfAll.columns:
        if '_based' in var:
            del dfAll[var]
            
    ###根据日期放置一个百分比顺序作为尝试
    dfAllPcentByDate = fe.pcent_by_other_col(dfAll,{'date':predictors},['ID']) 
    
    '''dfAll['date'] = pd.qcut(dfAll['date'],q=10,labels=['date_bin_%d'%i for i in range(10)])
    dfAll = pd.concat([dfAll,pd.get_dummies(dfAll['date'])],axis=1)'''
    ###目前不想使用日期，觉得用处不大
    del dfAll['date']

    #for minus in 
    
    '''for ratio in [['PartI_6','PartI_5'],['PartI_7','PartI_5'],['PartII_3','PartII_2'],['PartII_4','PartII_2'],['PartV_2','PartV_1'],['PartII_1','PartII_2']]:
        dfAll[ratio[0]+'_divided_'+ratio[1]] = dfAll[ratio[0]]/dfAll[ratio[1]]'''
        
    '''for multiply in [['PartV_1','PartV_9'],['PartV_1','PartV_10'],['PartV_1','PartV_11'],['PartV_1','PartV_12'],['PartV_1','PartV_13']]:
        dfAll[multiply[0]+'_multiply_'+multiply[1]] = dfAll[multiply[0]]/dfAll[multiply[1]]'''
    
    
    ###看看missing 的情况
    '''dfAll['missing_cnt'] = dfAll[check_missing].isnull().sum(axis=1)
    for i in range(len(check_missing)):
        dfAll['missing_part%d'%i] = dfAll[check_missing[i]].isnull().astype(int)'''
    
    
    '''for key,group in {'PartI':['PartI_1','PartI_2','PartI_3','PartI_4'],'PartII':['PartII_1','PartII_3','PartII_4']}.items():
        dfAll['group_'+key+'_std'] = dfAll[group].std(axis=1)
        dfAll['group_'+key+'_mean'] = dfAll[group].mean(axis=1)
        dfAll['group_'+key+'_median'] = dfAll[group].median(axis=1)'''
    
    #dfAll = dfAll.merge(dfAllPcentByOther,'inner','ID')
    dfAll = dfAll.merge(dfAllPcentByDate,'inner','ID')
    
    
    dfTrain = dfAll.loc[dfAll['ID'].isin(train['ID'])]
    dfPred = dfAll.loc[dfAll['ID'].isin(pred['ID'])]
    
    return dfTrain,dfPred,var_dict

In [4]:
dfTrain,dfPred,var_dict = FeatAll(dfTrain,dfPred)
predictors = dfPred.columns.tolist()
predictors.remove('ID')
predictors.remove('Y')

C:\Users\Leo Mao\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
len(predictors)

76

In [ ]:
predictors

In [ ]:
'''keep = ['ID']
for var in predictors:
    if 'pcent' in var and '_y' not in var:
        print(var)
        keep.append(var)
keep
dfAll = pd.concat([dfTrain[keep],dfPred[keep]])
rename_dict = {}
for var in keep:
    if '_x' in var:
        rename_dict[var] = var[0:-2]
dfAll = dfAll.rename(columns=rename_dict)
dfAll.to_csv('../../Cache/pcent_by_other.csv',index = False)'''

In [6]:
n_splits=5
test_result,result,imp = mm.xgb_kfold(dfTrain,dfPred,predictors,n_splits)

[0]	train-rmse:5.27715	eval-rmse:5.32362
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 10 rounds.
[100]	train-rmse:2.29632	eval-rmse:2.39389
[200]	train-rmse:1.44578	eval-rmse:1.61412
[300]	train-rmse:1.24854	eval-rmse:1.46511
[400]	train-rmse:1.18574	eval-rmse:1.43782
[500]	train-rmse:1.14792	eval-rmse:1.43083
Stopping. Best iteration:
[533]	train-rmse:1.13649	eval-rmse:1.42843

Best tree is 534, performance is 1.136486, 1.428427
[0]	train-rmse:5.29716	eval-rmse:5.24312
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 10 rounds.
[100]	train-rmse:2.32321	eval-rmse:2.26567
[200]	train-rmse:1.47999	eval-rmse:1.45247
[300]	train-rmse:1.2813	eval-rmse:1.30264
[400]	train-rmse:1.2141	eval-rmse:1.2804
[500]	train-rmse:1.17266	eval-rmse:1.27422
Stopping. Best iteration:
[496]	train-rmse:1.17421	eval-rmse:1.27413

Best tree is 

In [7]:
for i in range(1,n_splits+1):
    imp['imp_fold%d'%i] = imp['imp_fold%d'%i]/imp['imp_fold%d'%i].sum()
imp['sum_imp'] = imp[['imp_fold%d'%i for i in range(1,n_splits+1)]].sum(axis=1)

In [9]:
imp.sort_values('sum_imp',ascending=False)
tmpPredictor = imp.sort_values('sum_imp',ascending=False)['variable'].values.tolist()[0:40]

In [10]:
test_result,result,imp = mm.xgb_kfold(dfTrain,dfPred,tmpPredictor,n_splits)

[0]	train-rmse:5.27723	eval-rmse:5.32348
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 10 rounds.
[100]	train-rmse:2.29584	eval-rmse:2.39508
[200]	train-rmse:1.44841	eval-rmse:1.61365
[300]	train-rmse:1.25256	eval-rmse:1.46636
[400]	train-rmse:1.18811	eval-rmse:1.43747
[500]	train-rmse:1.15055	eval-rmse:1.42969
Stopping. Best iteration:
[492]	train-rmse:1.15333	eval-rmse:1.42969

Best tree is 493, performance is 1.153331, 1.429686
[0]	train-rmse:5.29711	eval-rmse:5.24313
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 10 rounds.
[100]	train-rmse:2.32031	eval-rmse:2.26536
[200]	train-rmse:1.47748	eval-rmse:1.45358
[300]	train-rmse:1.28247	eval-rmse:1.30593
[400]	train-rmse:1.21581	eval-rmse:1.28023
[500]	train-rmse:1.17711	eval-rmse:1.27443
Stopping. Best iteration:
[502]	train-rmse:1.17642	eval-rmse:1.27429

Best tree 

In [11]:
n_splits=5
other_note ='_test'
result['score']=result[['Score_%d'%i for i in range(1,n_splits+1)]].mean(axis=1)
submit = result[['ID','score']]
today = datetime.date.today().strftime('%Y-%m-%d')
result.to_csv('../../Submission/result/result_%s'%today+other_note+'.csv',index=False)
submit['score'].to_csv('../../Submission/submit_%s'%today+other_note+'.csv',header=False,index=False)
test_result.to_csv('../../Submission/test/test_result_%s'%today+other_note+'.csv',index=False)
imp.to_csv('../../Submission/imp/importance_%s'%today+other_note+'.csv',index=False)

In [ ]:
###基础变量
1.83977998784

###增加部分比例数据，已经缺失体检项目情况
1.84295123211  imp来看，missing信息没有意义

###上一版去除missing
1.83839589522



###增加一些比值信息
1.84354856365


###根据日期的百分比，效果更加不好了
1.85046680299  变量选择之后 1.83862284729